In [ ]:
import pandas as pd
import requests
from datetime import datetime
import pytz
import pymysql
from sqlalchemy import create_engine
import sqlalchemy as db
import mysql.connector
from pandas import json_normalize
import json

# Getting the Populations Data

In [ ]:
# i used an API method here insted of the data scraping i shared just for more accurate data 
import requests
import pandas as pd
import json
from pandas import json_normalize


def get_city_info(city_names):
    api_url = 'https://api.api-ninjas.com/v1/city'
    api_key = '045BPSxNWDo75G4L3M3A6A==1x3VPcWgTzu4EYt1'
    all_city_dataframes = []

    for name in city_names:
        params = {'name': name}
        headers = {'X-Api-Key': api_key}

        cities_response = requests.get(api_url, params=params, headers=headers)

        if cities_response.status_code == 200:
            cities_json = cities_response.json()
            df = pd.DataFrame(cities_json)
            all_city_dataframes.append(df)
    main_city_population = pd.concat(all_city_dataframes, ignore_index=True)
    return main_city_population

# Example usage:
city_names_list = ['Berlin', 'Paris', 'London']
get_city_info(city_names_list)

In [ ]:
city = get_city_info(city_names_list)

## inserting my data to MySQL

In [ ]:
def insert_dataframe_to_mysql(df, table):
    user='root'
    password='S0795735786s' #type your root password here
    host='127.0.0.1' #'127.0.0.1', # to connect to your local instance
    database='gans' #type the name of the database you want to use here
        

    # Establish a connection to the MySQL database using SQLAlchemy engine
    engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

    try:
        df.to_sql(table, con=engine, if_exists="append", index=False)
        print("succes !")
    except Exception as e:
        print(f"error : {e}")

insert_dataframe_to_mysql(city, "cities_and_populations")

In [ ]:
# setting a connection for MySQL
user='root'
password='S0795735786s' #type your root password here
host='127.0.0.1' #'127.0.0.1', # to connect to your local instance
database='gans' #type the name of the database you want to use here
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}', echo=True)
connection = engine.connect()

In [ ]:
# getting the data from my cities_and_populations in MySQL
query = "select * from cities_and_populations"
city_df = pd.read_sql(query, connection)
city_df = pd.DataFrame(city_df)
city_df

# Getting the Weather data 

In [ ]:
def weather_details(df):
    # creating dic for the weather 
    weather_dic = {"city_id":[],
                "country":[],
                "forcast_time":[],
                "temperature":[],
                "temperature_feels_like":[],
                "outlook":[],
                "outlook_description":[],
                "wind_speed":[],
                "wind_deg":[],
                "pressure":[],
                "humidity":[],
                "clouds":[],
                "rain":[],
                "snow":[]
                }

    for _, row in df.iterrows():
        API_key = "155ac89acb627b07f1c68cb640aef942"

    # making a loop for the cities
    
        city = row["name"]
        city_id = row["city_id"]
        weather_url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
        weather_response = requests.get(weather_url)
        weather_jason = weather_response.json()

        # getting the information fom the Jason
        for i in weather_jason["list"]:
            weather_dic["city_id"].append(city_id)
            weather_dic["country"].append(weather_jason["city"]["country"])
            weather_dic["forcast_time"].append(i["dt_txt"])
            weather_dic["temperature"].append(i["main"]["temp"])
            weather_dic["temperature_feels_like"].append(i["main"]["feels_like"])
            weather_dic["outlook"].append(i["weather"][0]["main"])
            weather_dic["outlook_description"].append(i["weather"][0]["description"])
            weather_dic["wind_speed"].append(i["wind"]["speed"])
            weather_dic["wind_deg"].append(i["wind"]["deg"])
            weather_dic["pressure"].append(i["main"]["pressure"])
            weather_dic["humidity"].append(i["main"]["humidity"])
            try:
                weather_dic["clouds"].append(i["clouds"]["all"])
            except:
                weather_dic["clouds"].append("0")
            try:
                weather_dic["snow"].append(i["snow"]["all"])
            except:
                weather_dic["snow"].append("0")
            try:
                weather_dic["rain"].append(i["rain"]["3h"])
            except:
                weather_dic["rain"].append("0")  
    return pd.DataFrame(weather_dic)


In [ ]:
weather_df = weather_details(city_df)
weather_df

In [ ]:
# inserting data to MySQL
insert_dataframe_to_mysql(weather_df, "weather")

# Getting the Airports Data


In [ ]:
def icao_code_for_airports(latitudes, longitudes):
    list_for_df=[]
    for index, value in enumerate(latitudes):
        airports_url = "https://aviation-reference-data.p.rapidapi.com/airports/search"

        querystring = {"lat":latitudes[index],"lon":longitudes[index],"radius":"50"}

        airports_headers = {
            "X-RapidAPI-Key": "94f5115229mshc91b32bba10ca7ap184966jsne87554a73ea6",
            "X-RapidAPI-Host": "aviation-reference-data.p.rapidapi.com"
        }

        airports_response = requests.get(airports_url, headers=airports_headers, params=querystring)
        
        list_for_df.append(pd.json_normalize(airports_response.json()))

    return  pd.concat(list_for_df, ignore_index=True)

In [ ]:
# the same result as before but here we have the city_column appended to the table
def icao_code_for_airports(df):
    city_id_df=[]
    list_for_df=[]
    for _,row in df.iterrows():
        latitude = row["latitude"]
        longitude = row["longitude"]
        city_id = row["city_id"]


        
           
        airports_url = "https://aviation-reference-data.p.rapidapi.com/airports/search"

        querystring = {"lat":latitude,"lon":longitude,"radius":"100"}

        airports_headers = {
            "X-RapidAPI-Key": "94f5115229mshc91b32bba10ca7ap184966jsne87554a73ea6",
            "X-RapidAPI-Host": "aviation-reference-data.p.rapidapi.com"
        }

        airports_response = requests.get(airports_url, headers=airports_headers, params=querystring).json()
        
        for i in range(len(airports_response)):
            airports = {"city_id":city_id, 
                        "iataCode":airports_response[i]["iataCode"],
                        "icaoCode":airports_response[i]["icaoCode"],
                        "name":airports_response[i]["name"],
                        "alpha2countryCode":airports_response[i]["alpha2countryCode"],
                        "latitude":airports_response[i]["latitude"],
                        "longitude":airports_response[i]["longitude"]}
            list_for_df.append(airports)            
                        





    return  pd.DataFrame(list_for_df)

In [ ]:
airports_data = icao_code_for_airports(city_df)
airports_data
indices_to_remove  = [0, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]
airports_data = airports_data.drop(indices_to_remove)

In [ ]:
#inserting the data to MySQL
insert_dataframe_to_mysql(airports_data, "airports")

In [ ]:
# setting a connection for MySQL
user='root'
password='S0795735786s' #type your root password here
host='127.0.0.1' #'127.0.0.1', # to connect to your local instance
database='gans' #type the name of the database you want to use here
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}', echo=True)
connection = engine.connect()

In [ ]:
# getting the data from my cities_and_populations in MySQL
query_airpots = "select * from airports"
airports_df1 = pd.read_sql(query_airpots, connection)
airports_df1 = pd.DataFrame(airports_df1)
airports_df1

# Getting the Flights Data

In [ ]:
from datetime import datetime, timedelta
def get_arrival_date(response, i):
    if 'arrivals' in response and len(response['arrivals']) > i and 'movement' in response['arrivals'][i] and 'actualTimeLocal' in response['arrivals'][i]['movement']:
        return response['arrivals'][i]['movement']['actualTimeLocal'].split(' ')[0]
    else:
        return response['arrivals'][i]['movement']['scheduledTimeLocal'].split(' ')[0]

def get_actual_arr_local_time(response, i):
    if 'arrivals' in response and len(response['arrivals']) > i and 'movement' in response['arrivals'][i] and 'actualTimeLocal' in response['arrivals'][i]['movement']:
        return response['arrivals'][i]['movement']['actualTimeLocal'].split(' ')[1].split('+')[0]
    else:
        return response['arrivals'][i]['movement']['scheduledTimeLocal'].split(' ')[1].split('+')[0]

def get_delay_time(response, i):
    if 'arrivals' in response and len(response['arrivals']) > i and 'movement' in response['arrivals'][i] and 'actualTimeLocal' in response['arrivals'][i]['movement']:
        return response['arrivals'][i]['movement']['actualTimeLocal'].split(' ')[1].split('+')[1]
    else:
        return response['arrivals'][i]['movement']['scheduledTimeLocal'].split(' ')[1].split('+')[1]
    
def flights_information(df):
    flights_data = []

    for _, row in df.iterrows():

        icao = row['icaoCode']
        iata = row['iataCode']
        airport_id = row['airport_id']
        tommorow_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')

        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tommorow_date}T11:00/{tommorow_date}T23:00"

        querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"true","withLocation":"false"}

        headers = {
            "X-RapidAPI-Key": '0b7cffd425mshbf932b1b5f7e633p187a96jsna5aedb4ce276',
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        responses = requests.get(url, headers=headers, params=querystring)
        
        
        
        if responses.status_code != 200: 
            print(f"Error - Status Code: {responses.status_code} at line{_}")
            print(f"Response Content: {responses.text}")
            print('Problem with status code')
            continue
            
        response = responses.json()
        

        for i in range(len(response['arrivals'])):

            output = {
                'airport_id': airport_id,
                'arrival_date' : get_arrival_date(response,i),
                'flight_number' : response['arrivals'][i]['number'],
                'airline' : response['arrivals'][i]['airline']['name'],
                'flight_status' : response['arrivals'][i]['status'],
                
                'scheduled_arr_local_time' : response['arrivals'][i]['movement']['scheduledTimeLocal'].split(' ')[1].split('+')[0],
                
                'actual_arr_local_time' : get_actual_arr_local_time(response, i),
              
                'scheduled_arr_UTC_time' : str(pd.to_datetime(response['arrivals'][i]['movement']['scheduledTimeUtc'])).split(' ')[1].split('+')[0],
                
                'delay_time' : get_delay_time(response, i)
                
            }

            flights_data.append(output)

    flights_df = pd.DataFrame(flights_data)
    flights_df['scheduled_arr_local_time'] = pd.to_datetime(flights_df['scheduled_arr_local_time']).dt.time
    flights_df['actual_arr_local_time'] = pd.to_datetime(flights_df['actual_arr_local_time']).dt.time
    flights_df['delay_time'] = flights_df['delay_time'].apply(lambda x: datetime.strptime(x, "%M:%S").strftime("%H:%M:%S"))

    return flights_df

In [ ]:
flights_data = flights_information(airports_df1)
flights_data

In [ ]:
insert_dataframe_to_mysql(flights_data, "flights")